<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/Widen00-Lab3/Lab3/Widen00_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Approximations**
**Joel Widén**

# **Abstract**



```
# This is formatted as code
```

Short summary of the lab report. State the objectives, methods used, main results and conlusions.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

# **Set up environment**

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

1. Function: L2 projection to pw linear approximation over mesh in 1D

Input: function f(x), 1D mesh with nodes (x0,...,xn)
Output: vector of L2 projection of f(x) evaluated in (x0,...,xn)
Test: verify accuracy and convergence rate against exact function

# **Method**

In [13]:
#L2 projection on pw to linear approximation over mesh in 1D


class loc2globmap:
  def __init__(self, A_map, b_map):
    self.A_map = A_map
    self.b_map = b_map

#Reference function
def f(x):
  fx = x**2
  return fx

#Input x, interval nr k, mesh and if it is the 0 or 1 polynomial in nr
def lin_lagrange_pol(x, mesh, k, nr):
  x_k = mesh[k+1]
  x_k_m1 = mesh[k]
  h_k = x_k - x_k_m1
  if nr == 0:
    lambdax =  (x_k - x)/h_k
  else:
    lambdax = (x- x_k_m1)/h_k
  return lambdax

def get_no_local_shape_functions(k):
  #since we have a linear basis V this is 2 for all elements
  q = 2
  return q

def get_local_to_global_map(k):
  A_map = np.array([[[k,k], [k,k+1]],[[k+1,k],[k+1,k+1]]])
  b_map = np.array([k,k+1])
  loc2glob = loc2globmap(A_map, b_map)
  return loc2glob

def integrate_vector(f, k, i, mesh):
  a = mesh[k]
  b = mesh[k+1]
  # Trapezoidal rule
  bi = f(a)*lin_lagrange_pol(a, mesh, k, i)*0.5*(b-a) + f(b)*lin_lagrange_pol(b, mesh, k, i)*0.5*(b-a)
  return bi

def integrate_matrix(k, i, j):
  #For constant mesh distance hk is the same for all
  hk = 0.1
  if i == j:
    aij = hk/3
  else:
    aij = hk/6
  return aij

def add_to_global_vector(b, b_k, loc2glob):
  print(b[loc2glob.b_map[0]:loc2glob.b_map[1]])
  b[loc2glob.b_map[0]:loc2glob.b_map[1]] = b[loc2glob.b_map[0]:loc2glob.b_map[1]] + b_k
  print(b)
  return b

#From algorithm 9.2
def assembly_system(f, A, b, mesh):
  no_elements = len(b) - 1
  for k in range(0, no_elements):
    q = get_no_local_shape_functions(k)
    loc2glob = get_local_to_global_map(k)
    b_k = np.zeros(q)
    a_k = np.zeros((q, q))
    for i in range(0, q):
      b_k[i] = integrate_vector(f, k, i, mesh)
      for j in range(0, q):
        a_k[i, j] = integrate_matrix(k, i, j)
    print(a_k)
    print(b_k)
    b = add_to_global_vector(b, b_k, loc2glob)
    #A = add_to_global_matrix(A, a_k, loc2glob)
  return A, b

def l2_to_pw(f, mesh):
  A = np.zeros((len(mesh), len(mesh)))
  b = np.zeros(len(mesh))

  A, b = assembly_system(f, A, b, mesh)
  return l2_to_pw


x_n = 1
interval = 0.1
mesh = np.arange(0 ,x_n+interval, interval)

f_approx = l2_to_pw(f, mesh)


[[0.03333333 0.01666667]
 [0.01666667 0.03333333]]
[0.     0.0005]
[0.]


ValueError: could not broadcast input array from shape (2,) into shape (1,)

# **Results**

# **Discussion**